In [1]:
import pyspark as ps
spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("yelp_academic") 
        .getOrCreate()
        )
sc = spark.sparkContext

In [2]:
biz = spark.read.json('yelp_dataset/yelp_academic_dataset_business.json')

In [3]:
rev = spark.read.json('yelp_dataset/yelp_academic_dataset_review.json')

In [5]:
user = spark.read.json('yelp_dataset/yelp_academic_dataset_user.json')

In [ ]:
biz.createOrReplaceTempView('yelp_business')
rev.createOrReplaceTempVie

In [ ]:
rev.cer